In [1]:
from tensorflow.keras.models import load_model

In [12]:
import mlflow
experiment_name = "test model"
run_name="test"


In [13]:
# model = load_model('keras_model.h5')
model = load_model("keras_Model.h5", compile=False)


In [14]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)            │ (None, 1280)                │         410,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_3 (Sequential)            │ (None, 6)                   │         128,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 538,908 (2.06 MB)

 Trainable params: 524,828 (2.00 MB)

 Non-trainable params: 14,080 (55.00 KB)

In [17]:
def create_experiment(experiment_name,run_name,model):
    import mlflow
    #mlflow.set_tracking_uri("http://localhost:5000") #uncomment this line if you want to use any database like sqlite as backend storage for model
    mlflow.set_experiment(experiment_name)
    
    with mlflow.start_run():
        
            
        
        mlflow.sklearn.log_model(model, "model")
        

        mlflow.set_tag("tag1", "trash model")
        mlflow.set_tags({"tag2":"CNN", "tag3":"Production"})
            
    print('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))

In [18]:
create_experiment(experiment_name,run_name,model)

2024/09/28 14:03:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run - test is logged to Experiment - test model


In [2]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

In [11]:


# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("platic bottle.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Class: plastic
Confidence Score: 0.9991473


In [9]:
# hack to change model config from keras 2->3 compliant
import h5py
f = h5py.File("keras_Model.h5", mode="r+")
model_config_string = f.attrs.get("model_config")
if model_config_string.find('"groups": 1,') != -1:
    model_config_string = model_config_string.replace('"groups": 1,', '')
    f.attrs.modify('model_config', model_config_string)
    f.flush()
    model_config_string = f.attrs.get("model_config")
    assert model_config_string.find('"groups": 1,') == -1

f.close()

In [18]:

import requests

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("platic bottle.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
import codecs, json 
# Load the image into the array
data[0] = normalized_image_array
data2 = data.tolist()
# json.dump(data2, codecs.open('data.json', 'w', encoding='utf-8'), 
#           separators=(',', ':'), 
#           sort_keys=True, 
#           indent=4)



In [19]:
import numpy as np
import requests
import json

# # Example numpy array (replace with your actual array)
# array = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])

# # Convert numpy array to list
# array_list = array.tolist()

# Prepare the payload
payload = {
    "inputs": data2
}

# Send the request
response = requests.post(
    url="http://127.0.0.1:9999/invocations",
    headers={"Content-Type": "application/json"},
    data=json.dumps(payload)
)

# Print the response
print(response.json())


{'predictions': [[0.0008376993355341256, 0.9991472959518433, 1.477425404239341e-10, 4.593506020000859e-09, 1.3855251381755807e-05, 1.2391130894684466e-06]]}


In [9]:
data[0]

array([[[-1.2597680e-01,  7.2587260e-43, -1.8383431e-01],
        [ 7.2587260e-43,  1.1333702e-41,  0.0000000e+00],
        [-1.4150727e-01,  7.2587260e-43,  1.4012985e-45],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 1.1353320e-41,  0.0000000e+00, -1.4157975e-01],
        [ 7.2587260e-43,  1.5722569e-42,  0.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00,  1.1353320e-41],
        [ 0.0000000e+00, -1.4158022e-01,  7.2587260e-43],
        [ 1.5736582e-42,  0.0000000e+00,  0.0000000e+00],
        ...,
        [ 7.2587260e-43,  1.5848686e-42,  0.0000000e+00],
        [-1.4162087e-01,  7.2587260e-43,  1.1018410e-41],
        [ 0.0000000e+00, -1.4161992e-01,  7.2587260e-43]],

       [[ 3.3631163e-44,  0.0000000e+00, -6.1465055e-10],
        [ 7.2587260e-43,  1.4012985e-45,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  2.7577554e-42],
        ...,
        [ 5.7453237e-44,  2.7605580e-42,  6.0255834e-44],
        [-8.3137672e-24,  4.5

In [6]:
inference_request

{'dataframe_records': array([[[[ 0.4666667 , -0.01176471, -0.3490196 ],
          [ 0.48235297, -0.00392157, -0.3490196 ],
          [ 0.45098042, -0.01960784, -0.38039213],
          ...,
          [-0.5372549 ,  0.22352946,  0.17647064],
          [-0.6156863 ,  0.12156868,  0.082353  ],
          [-0.70980394,  0.0196079 , -0.01960784]],
 
         [[ 0.45098042, -0.02745098, -0.36470586],
          [ 0.47450984, -0.01176471, -0.3490196 ],
          [ 0.4666667 , -0.01960784, -0.36470586],
          ...,
          [-0.58431375,  0.1686275 ,  0.14509809],
          [-0.36470586,  0.34901965,  0.3411765 ],
          [-0.5764706 ,  0.05098045,  0.082353  ]],
 
         [[ 0.43529415, -0.04313725, -0.38039213],
          [ 0.4666667 , -0.01176471, -0.3490196 ],
          [ 0.48235297, -0.00392157, -0.34117645],
          ...,
          [-0.77254903, -0.02745098, -0.04313725],
          [-0.6862745 ,  0.05098045,  0.03529418],
          [-0.7254902 ,  0.00392163,  0.00392163]],
 
       

In [21]:
#code to serve model
mlflow models serve --model-uri runs:/d4ec388e80d64096881eeb0424bcc373/model --port 9999 --env-manager local

SyntaxError: invalid syntax (2588343184.py, line 2)